#### Parse additional historical features from the EHR (medications, outpatient visits) and time-series data (blood tests, vital signs)

In [1]:
import pandas as pd
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.patches import Patch
from matplotlib.dates import DateFormatter
from datetime import timedelta, datetime
from tqdm import tqdm
import numpy as np
from scipy import stats, special
from tableone import TableOne

import os
import json
import re
import pprint
import missingno as msno
from statannotations.Annotator import Annotator
import warnings

pd.set_option('display.max_rows', None)

#### Load linked demographics data and helper-functions

In [ ]:
demo_data = pd.read_csv('../outputs/linked_data/linked_demographics.csv')
print(demo_data.shape, demo_data.subject_id.nunique())

In [2]:
core_path = '../../data/MIMIC-IV/mimiciv/3.1'
ed_path = '../../data/MIMIC-IV/mimic-iv-ed/3.1'
demo_path = '../outputs/linked_data/linked_demographics.csv'
ndc_path = '../../data/MIMIC-IV/config/NDC_product_table.csv'

In [ ]:
def dataframe_from_csv(path, compression='gzip', chunksize=None):
    return pd.read_csv(path, compression=compression,
                       chunksize=chunksize,
                       low_memory=False)

########################## MEDICATIONS ##########################
def read_prescriptions_table(mimic4_path, demo_path):
    meds = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/emar.csv.gz'))
    print('Loaded raw administrations table')
    linked_demo_data = pd.read_csv(demo_path)
    meds = meds.reset_index(drop=True)
    print(meds.subject_id.isnull().sum(), meds.shape)
    ### Link relevant medications and prepare for parsing
    meds = meds[meds.subject_id.isin(linked_demo_data.subject_id)]
    print(meds.subject_id.isnull().sum(), meds.shape)
    meds['charttime'] = pd.to_datetime(meds.charttime)
    meds = pd.merge(meds, linked_demo_data[['subject_id', 'admittime']], 
                    on='subject_id', how='left')
    print(meds.subject_id.isnull().sum(), meds.shape)
    meds = meds[meds.charttime <= meds.admittime]
    meds = meds.dropna(subset=['medication', 'event_txt'])
    print(meds.subject_id.isnull().sum(), meds.shape)
    ### Filter correctly administered medications
    meds = meds[meds.event_txt.isin(['Administered', 'Confirmed', 'Started'])]
    print(meds.subject_id.isnull().sum(), meds.shape)
    return meds[['subject_id', 'admittime', 'charttime', 'medication']]

def prepare_prescription_features(meds, demo_data, top_threshold=50):
    print('Getting prescription features')
    meds['medication'] = meds['medication'].str.lower().astype(str).str.strip().str.replace(' ', '_')
    meds = meds.sort_values(['subject_id', 'charttime'])
    top_meds = meds.medication.value_counts().head(top_threshold).index.tolist()
    #### Filter most common medications
    meds = meds[meds.medication.isin(top_meds)]
    meds['admittime'] = pd.to_datetime(meds['admittime'])
    ### Clean some of the top medication fields
    meds['medication'] = np.where(meds['medication'].str.contains('vancomycin'), 'vancomycin', meds['medication'])
    meds['medication'] = np.where(meds['medication'].str.contains('acetaminophen'), 'acetaminophen', meds['medication'])
    meds_ids = meds.groupby(['subject_id', 'medication', 'admittime']).size().reset_index(name='n_presc')
    meds_min = meds.groupby(['subject_id', 'medication', 'admittime'])['charttime'].min().reset_index(name='first_date')
    meds_min['first_date'] = pd.to_datetime(meds_min['first_date'])
    meds_max = meds.groupby(['subject_id', 'medication', 'admittime'])['charttime'].max().reset_index(name='last_date')
    meds_max['last_date'] = pd.to_datetime(meds_max['last_date'])
    meds_min['dsf'] = (meds_min['admittime'] - meds_min['first_date']).dt.days
    meds_max['dsl'] = (meds_max['admittime'] - meds_max['last_date']).dt.days
    meds_ids = pd.merge(meds_ids, meds_min[['subject_id', 'medication', 'dsf']], on=['subject_id', 'medication'], how='left')
    meds_ids = pd.merge(meds_ids, meds_max[['subject_id', 'medication', 'dsl']], on=['subject_id', 'medication'], how='left')
    meds_ids = meds_ids.drop(['admittime'], axis=1)
    #### Pivot table and create drug-specific features
    def rename(col):
        if isinstance(col, tuple):
            col = '_'.join(str(c) for c in col)
        return col
    meds_piv = meds_ids.set_index(['subject_id', 'medication']).unstack()
    meds_piv.columns = map(rename, meds_piv)
    meds_piv = meds_piv.reset_index()
    meds_piv_total = meds_ids.groupby(['subject_id'])['medication'].nunique().reset_index(name='total_n_presc')
    demo_data = pd.merge(demo_data, meds_piv_total, on='subject_id', how='left')
    demo_data = pd.merge(demo_data, meds_piv, on='subject_id', how='left')
    ### Fill missing values
    days_cols = [col for col in demo_data.columns if 'dsf' in col or 'dsl' in col]
    demo_data[days_cols] = demo_data[days_cols].fillna(9999).astype(np.int16)
    nums_cols = [col for col in demo_data.columns if 'n_presc' in col]
    demo_data[nums_cols] = demo_data[nums_cols].fillna(0).astype(np.int16)
    demo_data['total_n_presc'] = demo_data['total_n_presc'].fillna(0).astype(np.int8)
    return demo_data

def prepare_admin_features(poe, demo_data):
    #### Pivot table and create drug-specific features
    def rename(col):
        if isinstance(col, tuple):
            col = '_'.join(str(c) for c in col)
        return col
    print('Getting administration features')
    poe = poe[poe.subject_id.isin(demo_data.subject_id)]
    poe['ordertime'] = pd.to_datetime(poe['ordertime'])
    poe = poe.sort_values(['subject_id', 'ordertime'])
    demo_poe = pd.merge(demo_data, poe[['subject_id', 'ordertime', 'order_type']], on='subject_id', how='left')
    demo_poe = demo_poe[demo_poe['ordertime'] <= demo_poe['edregtime']]
    ### Filter order types of interest (can be extended to capture specific treatments)
    demo_poe = demo_poe[demo_poe.order_type.isin(['Nutrition', 'TPN', 'Cardiology', 'Radiology', 'Neurology', 'Respiratory', 'Hemodialysis'])]
    poe_ids = demo_poe.groupby(['subject_id', 'order_type']).size().reset_index(name='admin_proc_count')
    poe_piv = poe_ids.set_index(['subject_id', 'order_type']).unstack()
    poe_piv.columns = map(rename, poe_piv)
    poe_piv = poe_piv.reset_index()
    poe_piv_total = poe_ids.groupby(['subject_id'])['order_type'].nunique().reset_index(name='total_proc_count')
    demo_data = pd.merge(demo_data, poe_piv_total, on='subject_id', how='left')
    demo_data = pd.merge(demo_data, poe_piv, on='subject_id', how='left')
    ### Fill missing values
    nums_cols = [col for col in demo_data.columns if 'proc_count' in col]
    demo_data[nums_cols] = demo_data[nums_cols].fillna(0).astype(np.int16)
    return demo_data

def prepare_vitals(measures, ed_vitals, demo_data, output_path,
                   vitalsign_column_map = {
            "temperature": "Temperature",
            "heartrate": "Heart rate",
            "resprate": "Respiratory rate",
            "o2sat": "Oxygen saturation",
            "sbp": "Systolic blood pressure",
            "dbp": "Diastolic blood pressure"},
            vitalsign_uom_map = {
            "Temperature": "°F",
            "Heart rate": "bpm",
            "Respiratory rate": "insp/min",
            "Oxygen saturation": "%",
            "Systolic blood pressure": "mmHg",
            "Diastolic blood pressure": "mmHg",
            "BMI": "kg/m²"
        }):
    print('Getting time-series data for vitals')
    measures = measures[measures.subject_id.isin(demo_data.subject_id)]
    ed_vitals = ed_vitals[ed_vitals.subject_id.isin(demo_data.subject_id)]
    measures['chartdate'] = pd.to_datetime(measures['chartdate'] + ' ' + '00:00:00')
    measures = measures.rename(columns={'chartdate': 'charttime'})
    ed_vitals['charttime'] = pd.to_datetime(ed_vitals['charttime'])
    ed_vitals = ed_vitals.drop(['stay_id', 'pain', 'rhythm'], axis=1)
    measures = measures.drop(['seq_num'], axis=1)
    ### Prepare ed vitals time-series
    print('Preparing ED vitals for time-series data')
    ed_vitals = ed_vitals.merge(demo_data[['subject_id', 'edregtime']], on='subject_id', how='left')
    ed_vitals = ed_vitals[ed_vitals.charttime <= ed_vitals.edregtime]
    ed_vitals = ed_vitals.drop(['edregtime'], axis=1)
    ed_vitals = ed_vitals.rename(columns=vitalsign_column_map)
    ed_vitals = ed_vitals.melt(id_vars=['subject_id', 'charttime'], value_vars=[
                "Temperature",
                "Heart rate",
                "Respiratory rate",
                "Oxygen saturation",
                "Systolic blood pressure",
                "Diastolic blood pressure",
            ],
            var_name="label", value_name='value').sort_values(['subject_id', 'charttime'])
    ed_vitals['value'] = pd.to_numeric(ed_vitals['value'], errors='coerce')
    ed_vitals = ed_vitals.dropna(subset=['value'])
    ed_vitals['value_uom'] = ed_vitals['label'].map(vitalsign_uom_map)

    ### Prepare hospital measures time-series
    print('Preparing hospital measures for time-series data')
    measures = measures.merge(demo_data[['subject_id', 'edregtime']], on='subject_id', how='left')
    measures = measures[measures.charttime <= measures.edregtime]
    measures = measures.drop(['edregtime'], axis=1)
    measures['result_name'] = np.where(measures['result_name'].str.contains('Blood Pressure'), 'bp', measures['result_name'])
    measures[['result_sysbp', 'result_diabp']] = measures['result_value'].str.split('/', expand=True)
    measures['result_sysbp'] = pd.to_numeric(measures['result_sysbp'], errors='coerce')
    measures['result_diabp'] = pd.to_numeric(measures['result_diabp'], errors='coerce')
    measures['result_name'] = np.where(measures['result_name'].str.contains('BMI'), 'bmi', measures['result_name'])
    
    # Create separate rows for sysbp and diabp
    sysbp_measures = measures[['subject_id', 'charttime', 'result_sysbp']].rename(columns={'result_sysbp': 'value'})
    sysbp_measures['label'] = 'Systolic blood pressure'
    diabp_measures = measures[['subject_id', 'charttime', 'result_diabp']].rename(columns={'result_diabp': 'value'})
    diabp_measures['label'] = 'Diastolic blood pressure'

    # Concatenate the sysbp and diabp measures
    bp_measures = pd.concat([sysbp_measures, diabp_measures], axis=0)
    # Add BMI measurements
    bmi_measures = measures[measures['result_name'] == 'bmi'][['subject_id', 'charttime', 'result_value']].rename(columns={'result_value': 'value'})
    bmi_measures['label'] = 'BMI'
    measures = pd.concat([bp_measures, bmi_measures], axis=0)

    # Map the value_uom
    measures['value_uom'] = measures['label'].map(vitalsign_uom_map)

    # Combine with ed_vitals
    vitals = pd.concat([ed_vitals, measures], axis=0)
    vitals['linksto'] = 'vitals_measurements'
    vitals['value'] = pd.to_numeric(vitals['value'], errors='coerce')
    vitals = vitals.dropna(subset=['value'])
    # Drop any duplicate entries
    vitals = vitals.sort_values(['subject_id', 'charttime']).drop_duplicates(['subject_id', 'charttime', 'label'], keep='last')
    vitals.to_csv(output_path, index=False)
    return vitals

def get_generic_drugs(df, prod_table_path):
    """Takes NDC product table and prescriptions dataframe; adds column with NDC table's corresponding generic name"""
    mapping = pd.read_csv(prod_table_path, encoding='latin1')
    mapping['PRODUCTNDC'] = mapping['PRODUCTNDC'].apply(lambda x: '-'.join(x.split('-')[:2]))
    ndc_to_generic = mapping.set_index('PRODUCTNDC')['NONPROPRIETARYNAME'].to_dict()
    
    def brand_to_generic(ndc):
        matches = list(re.finditer(r"-", ndc))
        if len(matches) > 1:
            ndc = ndc[:matches[1].start()]
        return ndc_to_generic.get(ndc, np.nan)
    
    tqdm.pandas()
    df['generic_drug_name'] = df['ndc'].progress_apply(brand_to_generic)
    return df

In [ ]:
meds_data = read_prescriptions_table(core_path, demo_path)

In [ ]:
meds_feat = prepare_prescription_features(meds_data, demo_data)

In [ ]:
meds_feat.isnull().sum()

In [ ]:
meds_feat.total_n_presc.describe()

In [ ]:
for col in meds_feat.columns.tolist():
    if 'dsl' in col:
        print(meds_feat[col].value_counts().head(2))

In [52]:
meds_feat.to_csv('../outputs/linked_data/linked_meds_demographics.csv', index=False)

In [16]:
meds_feat = pd.read_csv('../outputs/linked_data/linked_meds_demographics.csv')

In [6]:
poe_test = dataframe_from_csv(os.path.join(core_path, 'hosp/poe.csv.gz'))

#### Specialty-related orders

In [ ]:
meds_feat = prepare_admin_features(poe_test, meds_feat)

In [25]:
meds_feat.to_csv('../outputs/linked_data/linked_meds_proc_demographics.csv', index=False)

In [ ]:
meds_feat.isnull().sum()

In [ ]:
meds_feat.shape

In [ ]:
for col in meds_feat.columns.tolist():
    if 'admin' in col:
        print(meds_feat[col].value_counts().head())
        print(meds_feat[col].describe())

#### Get vital signs and lab tests

In [40]:
measures_test = dataframe_from_csv(os.path.join(core_path, 'hosp/omr.csv.gz'))

In [ ]:
measures_test.isnull().sum()

In [ ]:
measures_test.head()

In [36]:
vitals_test = dataframe_from_csv(os.path.join(ed_path, 'ed/vitalsign.csv.gz'))

In [ ]:
vitals_test.isnull().sum()

In [ ]:
vitals_test.result_name.value_counts()

In [ ]:
measure_data = prepare_vitals(measures_test, vitals_test, meds_feat, '../outputs/linked_data/measures_ts.csv')

In [ ]:
print(measure_data.shape, measure_data.subject_id.nunique())
print(measure_data[measure_data.label.str.contains('blood pressure')].shape,
    measure_data[measure_data.label.str.contains('blood pressure')].subject_id.nunique())
print(measure_data[measure_data.label.str.contains('BMI')].shape,
    measure_data[measure_data.label.str.contains('BMI')].subject_id.nunique())
print(measure_data[measure_data.label.str.contains('Temperature')].shape,
    measure_data[measure_data.label.str.contains('Temperature')].subject_id.nunique())
print(measure_data[measure_data.label.str.contains('Respiratory rate')].shape,
    measure_data[measure_data.label.str.contains('Respiratory rate')].subject_id.nunique())
print(measure_data[measure_data.label.str.contains('Oxygen saturation')].shape,
    measure_data[measure_data.label.str.contains('Oxygen saturation')].subject_id.nunique())

##### Blood tests

In [4]:
demo_data = pd.read_csv('../outputs/linked_data/linked_demographics.csv')

In [18]:
#### Get most common lab tests as reference data
labs_lkup = pd.read_csv(os.path.join(core_path, 'hosp/labevents.csv'),
                        chunksize=10000000, low_memory=False)
labs_names = pd.read_csv(os.path.join(core_path, 'hosp/d_labitems.csv.gz'), compression='gzip')
labs_names = labs_names[['itemid', 'label']]
labs_lkup = pd.merge(labs_lkup.get_chunk(10000000), labs_names, on='itemid', how='left')
labs_lkup = labs_lkup.dropna(subset=['itemid', 'label', 'valuenum', 'valueuom'])
labs_lkup['label'] = labs_lkup['label'].str.lower().str.strip().str.replace(' ', '_').str.replace(',', '').str.replace('"', '')
labs_lkup = labs_lkup[['subject_id', 'itemid', 'label']].groupby(['itemid', 'label']).size().reset_index(name='n_tests')
labs_lkup = labs_lkup.sort_values(['n_tests'], ascending=False).head(50)
labs_lkup.to_csv('../outputs/linked_data/labs_lkup.csv', index=False)

In [ ]:
labs_lkup = pd.read_csv(os.path.join(core_path, 'hosp/labevents.csv'),
                        chunksize=10000, low_memory=False)
labs_lkup.get_chunk(10).charttime

In [19]:
### Export lab test ids as txt
lab_items_new = labs_lkup.itemid.tolist()
with open('../outputs/linked_data/lab_items.txt', 'w') as f:
    for item in lab_items_new:
        f.write("%s\n" % item)

In [ ]:
labs_data = pl.scan_csv(os.path.join(core_path, 'hosp/labevents.csv'), try_parse_dates=True)
d_items = (pl.read_csv(os.path.join(core_path, "hosp/d_labitems.csv.gz")).lazy().select(["itemid", "label"]))
# merge labitem id's with dict
labs_data = labs_data.join(d_items, how='left', on="itemid")
# select relevant columns
labs_data = (labs_data.select(["subject_id", "charttime", "itemid", "label", "value", "valueuom"])
        .with_columns(charttime=pl.col("charttime").cast(pl.Datetime), linksto=pl.lit("labevents")))
# get eligible lab tests prior to current episode
labs_data = labs_data.join(pl.from_pandas(demo_data[['subject_id', 'edregtime']]).lazy().
                           with_columns(edregtime=pl.col("edregtime").str.to_datetime(format="%Y-%m-%d %H:%M:%S")), 
                           how='left', on="subject_id")
labs_data = labs_data.filter(pl.col("charttime") <= pl.col("edregtime")).drop(["edregtime"])
# get most common items
labs_data = labs_data.filter(pl.col("itemid").is_in(set(lab_items_new)))
labs_data = labs_data.with_columns(
    pl.col("label").str.to_lowercase().str.replace(" ", "_").str.replace(",", "").str.replace('"', "").str.replace(" ", "_"),
    pl.col("charttime").str.replace("T", " ").str.strip_chars()
)
lab_events = labs_data.with_columns(
        value=pl.when(pl.col("value") == ".").then(None).otherwise(pl.col("value"))
)
lab_events = lab_events.with_columns(
    value=pl.when(pl.col("value").str.contains("_|<|ERROR"))
    .then(None)
    .otherwise(pl.col("value"))
    .cast(pl.Float64, strict=False)  # Attempt to cast to Float64, set invalid values to None
)
labs_data = labs_data.drop_nulls()

# Remove outliers using 2 std from mean
lab_events = lab_events.with_columns(mean=pl.col("value").mean().over(pl.count("label")))
lab_events = lab_events.with_columns(std=pl.col("value").std().over(pl.count("label")))
lab_events = lab_events.filter(
    (pl.col("value") < pl.col("mean") + pl.col("std") * 2)
    & (pl.col("value") > pl.col("mean") - pl.col("std") * 2)
).drop(["mean", "std"])

lab_events = lab_events.collect()
lab_events.write_csv(include_header=True, file='../outputs/linked_data/labs_ts.csv')

In [ ]:
labs_test = pd.read_csv('../outputs/linked_data/labs_ts.csv', chunksize=10)
labs_test.get_chunk(10).head()

In [ ]:
lab_events.head(10)

In [ ]:
print(labs_data.columns)

In [ ]:
labs_data.limit(10000).collect().head(10)